In [35]:
import bayesflow as bf
import pandas as pd
from itertools import product
from AmortizedMixture import *

In [2]:
@prior
def normal_mixture_prior(num_classes=2):
    return {
        'class_probabilities': np.random.dirichlet([1 for _ in range(num_classes)]),
        'class_parameters': np.sort(np.random.normal(size=num_classes))
    }

In [3]:
@simulator
def normal_mixture_simulator(params, num_obs=1000, num_classes=2):
    pi, mu = params.values()
    z = np.random.choice(a = [i for i in range(num_classes)], size = num_obs, replace = True, p=pi)
    loc = np.tile(mu[z], reps=(5, 1)).T
    y = np.random.normal(loc=loc, size=(num_obs, 5))[..., np.newaxis]
    print(y.shape)
    return {
        "latents": z,
        "observables": y
    }


In [4]:
model = bf.simulation.GenerativeModel(prior=normal_mixture_prior, simulator=normal_mixture_simulator)

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, )
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, )
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


(1000, 5, 1)
(1000, 5, 1)


In [5]:
obs = tf.convert_to_tensor([[[[1.1]], [[2]], [[3]]], [[[2.1]], [[1]], [[2]]]])
con = tf.convert_to_tensor([[1, 1.5], [2, 2.5]])
obs.shape

TensorShape([2, 3, 1, 1])

In [6]:
inference_net = IndependentClassificator(num_outputs=2)
local_summary_net = bf.networks.DeepSet(summary_dim=1)
global_summary_net = bf.networks.DeepSet(summary_dim=10)

global_summary_net(local_summary_net(obs))

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 1.9503919e-05, -1.6783171e-05,  4.0992268e-06,  2.4589286e-05,
         2.1820892e-05,  2.7740954e-05, -9.4840589e-06, -1.9195686e-05,
        -4.3674168e-06, -4.8987298e-05],
       [ 1.4434039e-05, -1.3813190e-05,  5.2439968e-06,  1.9209205e-05,
         1.7144599e-05,  2.3495608e-05, -9.2317214e-06, -1.6050204e-05,
        -3.7086763e-06, -4.0660172e-05]], dtype=float32)>

In [7]:
amortizer = AmortizedMixture(
    inference_net=inference_net,
    local_summary_net  = bf.networks.DeepSet(summary_dim=1),
    global_summary_net = bf.networks.DeepSet(summary_dim=10)
)
amortizer({'observables': obs})

<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[0.5000234 , 0.4999766 ],
        [0.50003654, 0.49996346],
        [0.50005114, 0.4999489 ]],

       [[0.50003684, 0.49996313],
        [0.5000208 , 0.49997917],
        [0.5000354 , 0.49996462]]], dtype=float32)>

In [8]:
amortizer.posterior_probs({'observables': obs})

<tf.Tensor: shape=(2, 3, 2), dtype=float32, numpy=
array([[[0.5000234 , 0.4999766 ],
        [0.50003654, 0.49996346],
        [0.50005114, 0.4999489 ]],

       [[0.50003684, 0.49996313],
        [0.5000208 , 0.49997917],
        [0.5000354 , 0.49996462]]], dtype=float32)>

In [18]:
lat = tf.one_hot([[1, 0, 1], [0, 0, 1]], depth=2)
lat
amortizer.compute_loss({'observables': obs, 'latents': lat})

<tf.Tensor: shape=(), dtype=float32, numpy=1.0362628>

In [17]:
model(5)

{'prior_non_batchable_context': None,
 'prior_batchable_context': None,
 'prior_draws': array([{'class_probabilities': array([0.70164115, 0.29835885]), 'class_parameters': array([0.80283069, 1.25142909])},
        {'class_probabilities': array([0.46539572, 0.53460428]), 'class_parameters': array([-0.33464622,  0.84031637])},
        {'class_probabilities': array([0.74191454, 0.25808546]), 'class_parameters': array([-0.48961011, -0.29484169])},
        {'class_probabilities': array([0.08528846, 0.91471154]), 'class_parameters': array([-0.82636079,  0.10977402])},
        {'class_probabilities': array([0.68423367, 0.31576633]), 'class_parameters': array([0.44186385, 0.8012559 ])}],
       dtype=object),
 'sim_non_batchable_context': None,
 'sim_batchable_context': None,
 'sim_data': array([{'latents': array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
               0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
               0, 1, 0, 0, 0, 1, 0,

In [ ]:
data = model(3)


In [ ]:
conf_data = configurator(data)

NameError: name 'configurator' is not defined

In [ ]:
conf_data['posterior_inputs']['summary_conditions'].shape

NameError: name 'conf_data' is not defined

In [ ]:
net = bf.inference_networks.InvertibleNetwork(num_params=1)

In [ ]:
d = np.array([[0, 2, 0], [0, 2, 3]])
d.shape

(2, 3)

In [ ]:
net(d.astype(np.float32), [[0], [0]])

(<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>)

In [13]:
ll = {"s": 2}
if ll.get("s"):
    print("ahoj")

'hello, ' + 2

ahoj


TypeError: can only concatenate str (not "int") to str

In [ ]:
posterior_amortizer = AmortizedPosterior()
likelihood_amortizer = AmortizedLikelihood()

posterior_likelihood_amortizer = JointAmortizer(amortizers = [posterior_amortizer, likelihood_amortizer])

In [ ]:
posterior_amortizer = AmortizedPosterior()
mixture_amortizer = AmortizedMixture()


# p(theta | s, y) x p(s | y) 
posterior_mixture_amortizer = ConditionalAmortizer(
    amortizer = posterior_amortizer,
    condition = mixture_amortizer
)

# p(s | theta, y) x p(theta | y)
mixture_posterior_amortizer = ConditionalAmortizer(
    amortizer = mixture_amortizer,
    condition = posterior_amortizer
)

In [104]:
x = np.ones(shape = (100, 50, 20, 10, 5, 3))

sn = bf.summary_networks.HierarchicalNetwork([
    bf.summary_networks.DeepSet(summary_dim=2),
    bf.summary_networks.DeepSet(summary_dim=4)
])

s = sn(x, return_all=True)
print([ss.shape for ss in s])

x = np.ones(shape = (100, 20, 20, 10, 5, 3))

s = sn(x, return_all=True)
print([ss.shape for ss in s])

[TensorShape([100, 50, 20, 10, 2]), TensorShape([100, 50, 20, 4])]
[TensorShape([100, 20, 20, 10, 2]), TensorShape([100, 20, 20, 4])]


In [89]:
x = np.ones(shape = (100, 50, 2))
sn = bf.summary_networks.TimeSeriesTransformer(input_dim=2)
sn(x)


<tf.Tensor: shape=(100, 10), dtype=float32, numpy=
array([[-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786978 ,
         0.4503148 , -0.17775209,  0.709754  ,  2.3236566 ,  1.0171022 ],
       [-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786978 ,
         0.4503148 , -0.17775209,  0.709754  ,  2.3236566 ,  1.0171022 ],
       [-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786978 ,
         0.4503148 , -0.17775209,  0.709754  ,  2.3236566 ,  1.0171022 ],
       [-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786978 ,
         0.4503148 , -0.17775209,  0.709754  ,  2.3236566 ,  1.0171022 ],
       [-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786978 ,
         0.4503148 , -0.17775209,  0.709754  ,  2.3236566 ,  1.0171022 ],
       [-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786978 ,
         0.4503148 , -0.17775209,  0.709754  ,  2.3236566 ,  1.0171022 ],
       [-1.0910923 , -2.2873282 , -2.68026   , -0.08268496, -0.5786

In [37]:
# Define your vectors (lists in Python)
nwords = [10, 20, 30, 40]
nobs = [1200, 1300, 1400, 1500]
lag = [1, 2]
version = [1, 2]

# Generate all combinations using itertools.product()
contexts = list(product(nwords, nobs, lag, version))
contexts = [{'nwords': nwords, 'nobs': nobs, 'lag': lag, 'version': version} for nwords, nobs, lag, version in contexts]
contexts

[{'nwords': 10, 'nobs': 1200, 'lag': 1, 'version': 1},
 {'nwords': 10, 'nobs': 1200, 'lag': 1, 'version': 2},
 {'nwords': 10, 'nobs': 1200, 'lag': 2, 'version': 1},
 {'nwords': 10, 'nobs': 1200, 'lag': 2, 'version': 2},
 {'nwords': 10, 'nobs': 1300, 'lag': 1, 'version': 1},
 {'nwords': 10, 'nobs': 1300, 'lag': 1, 'version': 2},
 {'nwords': 10, 'nobs': 1300, 'lag': 2, 'version': 1},
 {'nwords': 10, 'nobs': 1300, 'lag': 2, 'version': 2},
 {'nwords': 10, 'nobs': 1400, 'lag': 1, 'version': 1},
 {'nwords': 10, 'nobs': 1400, 'lag': 1, 'version': 2},
 {'nwords': 10, 'nobs': 1400, 'lag': 2, 'version': 1},
 {'nwords': 10, 'nobs': 1400, 'lag': 2, 'version': 2},
 {'nwords': 10, 'nobs': 1500, 'lag': 1, 'version': 1},
 {'nwords': 10, 'nobs': 1500, 'lag': 1, 'version': 2},
 {'nwords': 10, 'nobs': 1500, 'lag': 2, 'version': 1},
 {'nwords': 10, 'nobs': 1500, 'lag': 2, 'version': 2},
 {'nwords': 20, 'nobs': 1200, 'lag': 1, 'version': 1},
 {'nwords': 20, 'nobs': 1200, 'lag': 1, 'version': 2},
 {'nwords'